In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False,
                                               split_noise=True, n_fabricate_noise = 1600,
                                               split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm_random_input_modify import *

In [4]:
graph_name = 'SpectrogramMultiLSTMRandomInputModify'  # Specify graph class name

g_cnfg = Config('graph_04_kindof_finer_layers')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

#g_cnfg.conv1_n_filters = 16
#g_cnfg.conv1_kernel_size = 5
#g_cnfg.conv1_mp_size = 5
#g_cnfg.conv1_mp_strides = 2

#g_cnfg.conv2_n_filters = 32
#g_cnfg.conv2_kernel_size = 3
#g_cnfg.conv2_mp_size = 3
#g_cnfg.conv2_mp_strides = 2

g_cnfg.lstm_state_sizes = [128, 256, 512]
#g_cnfg.lstm_dropout = {'in': 'y', 'out': 'n', 'state': 'n'}

#g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.n_conv_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters
#g_cnfg.n_flat = g_cnfg.n_conv_flat + g_cnfg.lstm_state_sizes[-1]

g_cnfg.n_hidden1 = 256
g_cnfg.n_hidden2 = 128

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 1.0  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label(['_background_noise_'])])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [ ]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_04_kindof_finer_layers
X_img_h: 128
X_img_w: 71
Y_vector_len: 31
lr_decay_rate: 1.0
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [128, 256, 512]
n_hidden1: 256
n_hidden2: 128
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8

name: run_01
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 379
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramMultiLSTMRandomInputModify_graph_04_kindof_finer_layers_run_01
Training starts @ 12/24/2017 12:36:13
Epoch 1 Step 1 Best model saved @ 12/24/2017 12:36:20 [Ave valid] 3.434
Epoch 1 Step 1 ends @ 12/24/2017 12:36:20 [Train] 3.433, 6.7% [Valid] 3.5% [Ave valid] 3.434
Epoch 1 Step 50 Best model saved @ 12/24/2017 12:36:34 [Ave valid] 3.431
Epoch 1 Step 150 Best model sa

Epoch 16 Step 6,064 ends @ 12/24/2017 13:05:45 [Train] 0.082, 92.9% [Valid] 87.1% [Ave valid] 0.302
Epoch 17 Step 6,443 ends @ 12/24/2017 13:07:32 [Train] 0.074, 92.9% [Valid] 82.2% [Ave valid] 0.333
Epoch 18 Step 6,800 Best model saved @ 12/24/2017 13:09:14 [Ave valid] 0.275
Epoch 18 Step 6,822 ends @ 12/24/2017 13:09:23 [Train] 0.058, 100.0% [Valid] 88.8% [Ave valid] 0.290
Epoch 19 Step 7,200 Best model saved @ 12/24/2017 13:11:09 [Ave valid] 0.260
Epoch 19 Step 7,201 ends @ 12/24/2017 13:11:13 [Train] 0.044, 100.0% [Valid] 92.1% [Ave valid] 0.264
Epoch 20 Step 7,580 Best model saved @ 12/24/2017 13:13:00 [Ave valid] 0.259
Epoch 20 Step 7,580 ends @ 12/24/2017 13:13:00 [Train] 0.035, 100.0% [Valid] 92.6% [Ave valid] 0.259
Epoch 21 Step 7,600 Best model saved @ 12/24/2017 13:13:09 [Ave valid] 0.257
Epoch 21 Step 7,650 Best model saved @ 12/24/2017 13:13:23 [Ave valid] 0.257
Epoch 21 Step 7,750 Best model saved @ 12/24/2017 13:13:51 [Ave valid] 0.257
Epoch 21 Step 7,800 Best model save